In [1]:
##Estimación de indicadores de población indígena en la Ciudad de México
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

Loading required package: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
url<-"https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2024_trim2_csv.zip"


In [4]:
# Descargar el archivo y descomprimirlo
destfile <- "data/conjunto_de_datos_enoe_2024_2t_csv.zip"
download.file(url, destfile = destfile)
unzip(destfile, exdir = "data")

In [5]:
#Leer tabla y filtrar ocupados

ocupados_cdmx<-read_csv("data/ENOE_SDEMT224.csv", show_col_types = FALSE)%>%
janitor::clean_names()%>%
filter(ent == 9,
       clase2==1,
       ((eda>=15 & eda<=98 & !is.na(eda)) &
                                     (c_res==1 | c_res==3)))
nrow(ocupados_cdmx)


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] 4907

In [6]:
#Traer clasificación SINCO

sinco<-read_csv("data/ENOE_COE1T224.csv", show_col_types = FALSE)%>%janitor::clean_names()%>%
select(tipo, mes_cal,cd_a,ent,con,v_sel,n_hog,h_mud, n_ren,p3)%>%
#Renombrar p3 a sinco
rename(sinco = p3)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [7]:
#Unir tablas
ocupados_cdmx<-left_join(ocupados_cdmx, sinco, by = c("tipo", "mes_cal", "cd_a", "ent", "con", "v_sel", "n_hog", "h_mud", "n_ren"))%>%
#Clasificar empleo cultural usando la clasificación SINCO
mutate(cultura=case_when(sinco %in% c(2175,7611,2552,2131,7313,7342,7321,2174,7343,2173,2171,1721,1722,
2544,2271,2625,2162,1422,1322,1421,2541,2543,2542,2164,2151,2163,2144,2425,2655,3222,7322,2712,
7223,2551,5212,2636,4233,2172,8155,8144,7999,7344,2553,7352,2152,2161,7323,7201,7601,7401,7301,
2652,7332,9713,4312,7614,7353) ~ 1,
         TRUE ~ 0))
nrow(ocupados_cdmx)

[1] 4907

In [8]:
#Opción para ajustar si se tiene una sola upm
options(survey.lonely.psu="certainty")
dm<-ocupados_cdmx%>%
as_survey_design(ids = upm,
                 strata = est_d_tri,
                 weights = fac_tri)

In [9]:
#Total de ocupados en cdmx
dm%>%
summarise(ocupados = survey_total(vartype = "cv"))

ocupados,ocupados_cv
<dbl>,<dbl>
4834239,0.01735656


In [10]:
#Estimación de ocupados en cultura
print("Población ocupada en cultura")
dm%>%group_by(cultura)%>%
summarise(ocupados = survey_total(vartype = "cv"),
porcentaje = survey_mean(vartype = "cv"))%>%
mutate(porcentaje = porcentaje*100)%>%
filter(cultura==1)

[1] "Población ocupada en cultura"


cultura,ocupados,ocupados_cv,porcentaje,porcentaje_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,266496,0.07635535,5.512677,0.07551563
